# 3일차 과제 

- 아래 문제는 3일차 수업에 사용한 데이터를 활용하여 시험 문제와 유사한 형태로 문제를 만든 것입니다.
- 수업 내용과 교재를 참고하여 실제 시험을 보는 것처럼 한 셀 안에 코드를 작성해보세요.
- 제출 방법: 6/1(수)까지 코드를 작성한 .ipynb 파일을 yi.kyeongsuk@gmail.com로 제출 (***단, 코드 실행 시 정답 csv 파일이 생성되어야 합니다.***)


In [1]:
# 이 셀을 먼저 실행한 후 문제를 풀어주세요.
!git clone https://github.com/kyeongsuk-yi/data_analyst/

Cloning into 'data_analyst'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 26 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (26/26), done.


### 문제
아래는 boston housing 데이터 세트를 변형시킨 데이터이다.   

1) 제공 데이터 목록    
*   housing_y_train.csv : 주택 고가 여부 데이터(학습용), csv 형식의 파일
*   housing_X_train.csv, housing_X_test.csv : 주택 정보(학습 및 평가용), csv 형식의 파일

2) 주어진 학습용 데이터 (housing_X_train.csv, housing_y_train.csv)를 활용하여 해당 주택이 고가인지 아닌지를 예측하는 모형을 만든 후, 이를 평가용 데이터(housing_X_test.csv)에 적용하여 얻는 주택 고가 여부(고가일 경우 1, 아니면 0)를 다음과 같은 형식의 csv 파일로 생성하시오. (제출한 모델의 성능은 ROC-AUC 평가 지표에 따라 채점).

|isHighValue|
|---|
|0|
|0|
|1|
|1|
|0|
|...|

3) 유의사항
- 성능이 우수한 예측모형을 구축하기 위해서는 적절한 데이터 전처리, feature engineering, 분류 알고리즘 사용, 초매개변수 최적화, 모형 앙상블 등이 수반되어야 한다. 
- 이름.csv (예: 홍길동.csv) 파일이 만들어지도록 코드를 제출한다.
- 제출한 모델의 성능은 ROC-AUC 평가 지표에 따라 채점한다. 

In [21]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
# import pandas as pd
# X_test = pd.read_csv("./data_analyst/data/housing_X_test.csv")
# X_train = pd.read_csv("./data_analyst/data/housing_X_train.csv")
# y_train = pd.read_csv("./data_analyst/data/housing_y_train.csv")


######### 여기에 코드를 작성해 주세요. #########
# import packages

import pandas as pd
import numpy as np 

import warnings 
warnings.filterwarnings('ignore')

pd.set_option('expand_frame_repr', False)

X_test = pd.read_csv("./data_analyst/data/housing_X_test.csv")
X_train = pd.read_csv("./data_analyst/data/housing_X_train.csv")
y_train = pd.read_csv("./data_analyst/data/housing_y_train.csv")

# print(X_test.shape, X_train.shape, y_train.shape, '\n')
# print(X_test)
# print(X_train)
# print(y_train)

# print(X_test.isnull().sum(), '\n')
# print(X_train.isnull().sum()/len(X_train), '\n')
# print(y_train.isnull().sum(), '\n')

# print(X_train.head(1).T)
# print('\n')
# print(X_train.info())

del_cols = ['CHAS', 'TAX']
num_cols = ['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'PTRATIO', 'B', 'LSTAT']
cat_cols = []
y_col = ['isHighValue']

# LSTAT 변수 결측치 처리 
# 1) 결측이 존재하는 행 삭제 -> 이 때는 y_train 도 같이 지워야 한다 (X_test는 건들면 x)
# 2) sklearn.impute - SimpleImputer
# 3) fillna

med = X_train['LSTAT'].median()
X_train['LSTAT'] = X_train['LSTAT'].fillna(med)
# print(X_train.isnull().sum())

X_train.drop(del_cols, axis=1, inplace=True)
X_test.drop(del_cols, axis=1, inplace=True)

# 다중공선성체크 -> 보통 독립변수끼리 서로 상관관계가 0.8~0.9 이상 되면 처리를 해줍니다. (0.7은 조금 애매)
# print(X_train.describe())
# corr_df = X_train.corr()
# print(corr_df[corr_df>0.8])

# 3. 데이터 모형 구축
# - 데이터 분할
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=123)

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_tr[num_cols])
X_tr[num_cols] = sc.transform(X_tr[num_cols])
X_val[num_cols] = sc.transform(X_val[num_cols])
X_test[num_cols] = sc.transform(X_test[num_cols])

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier 

rf = RandomForestClassifier(max_depth=20, min_samples_leaf=2, random_state=123)
xgb1 = XGBClassifier(n_estimators=100, max_depth=10)
xgb2 = XGBClassifier(n_estimators=150, max_depth=10, min_child_weight=2)

rf.fit(X_tr, y_tr)
xgb1.fit(X_tr, y_tr)
xgb2.fit(X_tr, y_tr)

# 4. 모형 평가 
from sklearn.metrics import roc_auc_score 
pred_rf_tr = rf.predict_proba(X_tr)[:, 1]
pred_rf_val = rf.predict_proba(X_val)[:, 1]
pred_xgb1_tr = xgb1.predict_proba(X_tr)[:, 1]
pred_xgb1_val = xgb1.predict_proba(X_val)[:, 1]
pred_xgb2_tr = xgb2.predict_proba(X_tr)[:, 1]
pred_xgb2_val = xgb2.predict_proba(X_val)[:, 1]

score_rf_tr = roc_auc_score(y_tr, pred_rf_tr)
score_rf_val = roc_auc_score(y_val, pred_rf_val)
score_xgb1_tr = roc_auc_score(y_tr, pred_xgb1_tr)
score_xgb1_val = roc_auc_score(y_val, pred_xgb1_val)
score_xgb2_tr = roc_auc_score(y_tr, pred_xgb2_tr)
score_xgb2_val = roc_auc_score(y_val, pred_xgb2_val)

# print('rf: ', score_rf_tr, ' / ', score_rf_val)
# print('xgb1: ', score_xgb1_tr, ' / ', score_xgb1_val)
# print('xgb2: ', score_xgb2_tr, ' / ', score_xgb2_val)

pred = xgb1.predict_proba(X_test)[:, 1]
pd.DataFrame({'isHighValue': pred}).to_csv('1234567.csv', index=False)

################################################


# 답안 제출 참고
# 아래 코드 예측변수와 이름을 개인별로 변경하여 활용
# pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('홍길동.csv', index=False)